In [ ]:
import os
import copy
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import arch 
from arch import arch_model
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
sp500 = pd.read_csv('SP500_19900102_20221231.csv')[['caldt', 'vwretd']].set_index('caldt')


In [ ]:
# 设置滚动窗口的大小和步长，window_size分别设置为126、500、1000、2000，再分别保存到对应位置，csv文件后缀修改为对应的窗口大小
window_size = 2000
step_size = 1

# 存储预测结果的列表
outsampleprediction = []
for i in range(window_size, len(sp500), step_size):
    # 获取当前窗口的数据
    window_data = sp500.iloc[i - window_size:i]
    scale_mean = window_data.values.mean()
    scale_std = window_data.values.std()
    series_scaled = (window_data - scale_mean) / scale_std
    # ARIMA模型的预测
    arima_model = ARIMA(series_scaled, order=(1, 0, 1))
    arima_result = arima_model.fit()
    outsamplepredict = arima_result.forecast(steps=1)
    arima_predict = outsamplepredict.values[0] * scale_std + scale_mean
    # 预测ARIMA模型的残差
    residuals = arima_result.resid
    residuals = residuals * scale_std + scale_mean
    # 对残差进行GARCH(1,1)建模
    garch_model = arch_model(residuals, vol='Garch', p=1, q=1)
    results_garch = garch_model.fit()
    # 生成GARCH模型的样本外预测
    garch_forecast = results_garch.forecast(horizon=1)
    mean_forecast = garch_forecast.mean.iloc[-1, 0]
    # 计算样本外预测值
    outsample_predict = arima_predict + mean_forecast

    # 将预测结果添加到列表中
    outsampleprediction.append(outsample_predict)
    print(f"Predicting Day {i}...")

# 将预测结果添加到原始数据中
sp500['outpredictions'] = [None] * window_size + outsampleprediction

In [ ]:
sp500.to_csv('arima-garch.results/arima-garch2000.csv', index=True)